In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
config = {
    'domain': 'spouse', # NOTE: hardcode below still
#     'db_name': 'babble_spouse_demo',
#     'babbler_candidate_split': 1,
#     'babbler_label_split': 1,
}

In [3]:
# Get DB connection string and add to globals
# NOTE: $SNORKELDB must be set before any snorkel imports
import os

default_db_name = 'babble_' + config['domain'] + ('_debug' if config.get('debug', False) else '')
DB_NAME = config.get('db_name', default_db_name)
if 'postgres' in config and config['postgres']:
    DB_TYPE = 'postgres'
else:
    DB_TYPE = 'sqlite'
    DB_NAME += '.db'
DB_ADDR = "localhost:{0}".format(config['db_port']) if 'db_port' in config else ""
os.environ['SNORKELDB'] = '{0}://{1}/{2}'.format(DB_TYPE, DB_ADDR, DB_NAME)
print("$SNORKELDB = {0}".format(os.environ['SNORKELDB']))

$SNORKELDB = sqlite:///babble_spouse.db


In [4]:
from snorkel import SnorkelSession
session = SnorkelSession()

# Resolve config conflicts (nb_config > local_config > global_config)
from snorkel.contrib.babble.pipelines import merge_configs, get_local_pipeline
config = merge_configs(config)

from snorkel.models import candidate_subclass
candidate_class = candidate_subclass(config['candidate_name'], config['candidate_entities'])

Overwriting domain=None to domain=spouse
Overwriting print_freq=1 to print_freq=5
Overwriting LF_acc_prior_weight_default=1.0 to LF_acc_prior_weight_default=0.5
Overwriting decay=0.95 to decay=0.99
Overwriting init_class_prior=0 to init_class_prior=-1.15
Overwriting reg_param=0.1 to reg_param=0.5
Overwriting babbler_candidate_split=1 to babbler_candidate_split=[0, 1, 2]
Overwriting disc_model_class=lstm to disc_model_class=logreg


In [5]:
# user_lists = {
#     'spouse':  ['spouse', 'wife', 'husband', 'ex-wife', 'ex-husband'],
#     'family':  ['father', 'father', 'mother', 'sister', 'sisters', 
#                 'brother', 'brothers', 'son', 'sons', 'daughter', 'daughters',
#                 'grandfather', 'grandmother', 'uncle', 'uncles', 'aunt', 'aunts', 
#                 'cousin', 'cousins'],
#     'other':  ['boyfriend', 'girlfriend', 'boss', 'employee', 'secretary', 'co-worker']}

In [6]:
# import os
# import bz2
# import cPickle

# ctd_pickle_path = os.path.join(os.environ['SNORKELHOME'], 
#                                'tutorials/cdr/data/ctd.pkl.bz2') 
# print('Loading canonical ID ontologies...')
# with bz2.BZ2File(ctd_pickle_path, 'rb') as ctd_f:
#     ctd_unspecified, ctd_therapy, ctd_marker = cPickle.load(ctd_f)
# print('Finished loading canonical ID ontologies.')

# user_lists = {
#     'uncertain': ['combin', 'possible', 'unlikely'],
#     'causal': ['causes', 'caused', 'induce', 'induces', 'induced', 'associated with'],
#     'treat': ['treat', 'effective', 'prevent', 'resistant', 'slow', 'promise', 'therap'],
#     'procedure': ['inject', 'administrat'],
#     'patient': ['in a patient with', 'in patients with'],
#     'weak': ['none', 'although', 'was carried out', 'was conducted', 'seems', 
#              'suggests', 'risk', 'implicated', 'the aim', 'to investigate',
#              'to assess', 'to study'],
#     'ctd_unspecified': ctd_unspecified,
#     'ctd_therapy': ctd_therapy,
#     'ctd_marker': ctd_marker,
# }

In [7]:
user_lists = {}

In [8]:
from snorkel.contrib.babble import SemanticParser
semparser = SemanticParser(
    mode='text', candidate_class=candidate_class, user_lists=user_lists, beam_width=10)

Created grammar with 596 rules


In [9]:
from snorkel.contrib.babble import Explanation

explanations = [
    Explanation(
        name='LF1',
        condition="""person2 occurs in a phrase surrounded by quotes""",
        candidate='2a339c82-5086-40e8-91ba-e705418acc0d::span:2635:2636~~2a339c82-5086-40e8-91ba-e705418acc0d::span:2668:2683',
        label=False,
        semantics=None), # TBD
]

In [10]:
from snorkel.contrib.babble import link_explanation_candidates

candidates = session.query(candidate_class).all()
print(len(candidates))
explanations = link_explanation_candidates(explanations, candidates)

27688
Building list of target candidate ids...
Collected 1 unique target candidate ids from 1 explanations.
Gathering desired candidates...
Found 1/1 desired candidates
Linking explanations to candidates...
Linked 1/1 explanations


In [11]:
parses = semparser.parse(explanations, return_parses=True)
print(len(parses))

[========================================] 100%

0


In [ ]:
c = parses[0].explanation.candidate
print(c[0].get_span(), c[1].get_span())
print(c.get_parent().text)
print("")

for parse in parses:
    if parse.function(parse.explanation.candidate):
        print(semparser.grammar.translate(parse.semantics))
        print("")
        print(parse.semantics)
    else:
        print("FAIL!")
    print("")

In [ ]:
# semparser.grammar.print_chart()